In [20]:
import pandas as pd
import seaborn as sns
import numpy as np
df = pd.read_csv("C://Users//Chan Li Long//Documents//Protein//pdb_data_seq.csv")


In [21]:
df_labels = pd.read_csv("C://Users//Chan Li Long//Documents//Protein//pdb_data_no_dups.csv")
id2class = {k:v for k,v in zip(df_labels["structureId"],df_labels["classification"])}

In [22]:
df["class"] = df["structureId"].apply(lambda x : id2class[x])
count_df = df[["class","macromoleculeType"]].groupby("class").agg('count')
count_df = count_df.sort_values("macromoleculeType")
top_classes = list(count_df.index[-35:])
top_classes = [x for x in top_classes if x not in ["UNKNOWN FUNCTION",'STRUCTURAL GENOMICS, UNKNOWN FUNCTION']]

df = df[df["sequence"].apply(lambda x: isinstance(x,str))]
df = df[df["sequence"].apply(lambda x: (len(x)<=512) & (len(x)>=10))]
df = df[df["class"].apply(lambda x: x in top_classes)]
max_len = df["sequence"].apply(len).max()
df["sequence"] = df["sequence"].apply(list)
def pad(x,l=512):
    p = np.ones((1,512),dtype=str)
    p[:,0:len(x)] = x
    return p
df["sequence"] = df["sequence"].apply(lambda x: pad(x,max_len))
df

,structureId,chainId,sequence,residueCount,macromoleculeType,class
2,101D,A,"[[C, G, C, G, A, A, T, T, C, G, C, G, 1, 1, 1,...",24,DNA,DNA
3,101D,B,"[[C, G, C, G, A, A, T, T, C, G, C, G, 1, 1, 1,...",24,DNA,DNA
5,102D,A,"[[C, G, C, A, A, A, T, T, T, G, C, G, 1, 1, 1,...",24,DNA,DNA
6,102D,B,"[[C, G, C, A, A, A, T, T, T, G, C, G, 1, 1, 1,...",24,DNA,DNA
9,103D,A,"[[G, T, G, G, A, A, T, G, G, A, A, C, 1, 1, 1,...",24,DNA,DNA
...,...,...,...,...,...,...
467282,9MHT,C,"[[C, C, A, T, G, C, G, C, T, G, A, C, 1, 1, 1,...",351,Protein#DNA,TRANSFERASE/DNA
467283,9MHT,D,"[[G, T, C, A, G, N, G, C, A, T, G, G, 1, 1, 1,...",351,Protein#DNA,TRANSFERASE/DNA
467285,9NSE,A,"[[S, R, A, P, A, P, A, T, P, H, A, P, D, H, S,...",888,Protein,OXIDOREDUCTASE
467286,9NSE,B,"[[S, R, A, P, A, P, A, T, P, H, A, P, D, H, S,...",888,Protein,OXIDOREDUCTASE


In [23]:
all_seq = np.vstack(df["sequence"].values)

In [32]:
unique_seq = np.unique(all_seq)
unique_seq.sort()
vocab_dict = {x:i for i,x in enumerate(unique_seq)}
# vocab_dict

In [33]:
classes = df["class"].unique()
classes.sort()
class_dict = {x:i for i,x in enumerate(classes)}
# class_dict

In [35]:
convert = np.vectorize(lambda x : vocab_dict[x])
seq_int = convert(all_seq)

In [38]:
class_int = df["class"].apply(lambda x: class_dict[x]).values
class_int

array([ 3,  3,  3, ..., 14, 14,  5], dtype=int64)

In [40]:
data_dict = {}
data_dict["seq"] = seq_int
data_dict["class"] = class_int
data_dict["vocab"] = vocab_dict
data_dict["class_dict"] = class_dict

In [42]:
np.save("protein_processed.npy",data_dict)